In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %cd ./drive/MyDrive/Colab\ Notebooks/NLP_Project/

/content/drive/MyDrive/Colab Notebooks/NLP_Project


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
import random as rnd

In [4]:
def get_vocab(vocab_path, tags_path):
    vocab = {}
    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i  # to avoid the 0
        # loading tags (we require this to map tags to their indices)
    vocab['<PAD>'] = len(vocab) # 35180
    tag_map = {}
    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i

    return vocab, tag_map

def get_params(vocab, tag_map, sentences_file, labels_file):
    sentences = []
    labels = []

    with open(sentences_file) as f:
        for sentence in f.read().splitlines():
            # replace each token by its index if it is in vocab
            # else use index of UNK_WORD
            s = [vocab[token] if token in vocab
                 else vocab['UNK']
                 for token in sentence.split(' ')]
            sentences.append(s)

    with open(labels_file) as f:
        for sentence in f.read().splitlines():
            # replace each label by its index
            s = sentence.split(' ')
            # remove empty strings
            s = list(filter(None, s))
            l = [tag_map[label] for label in s] # I added plus 1 here
            labels.append(l)
    return sentences, labels, len(sentences)


# Importing and discovering the data

In [6]:
vocab, tag_map = get_vocab('./Dataset/characters/unique_chars.txt', './Dataset/characters/unique_labels.txt')
t_sentences, t_labels, t_size = get_params(vocab, tag_map, './Dataset/characters/t_chars.txt', './Dataset/characters/t_labels.txt')
v_sentences, v_labels, v_size = get_params(vocab, tag_map, './Dataset/characters/v_chars.txt', './Dataset/characters/v_labels.txt')
test_sentences, test_labels, test_size = get_params(vocab, tag_map, './Dataset/characters/test_chars.txt', './Dataset/characters/test_labels.txt')

In [7]:
# vocab translates from a word to a unique number
print('vocab["الأعم"]:', vocab["الأعم"])
# Pad token
print('padded token:', vocab['<PAD>'])

vocab["الأعم"]: 103799
padded token: 108278


In [8]:
# The possible tags
print(tag_map)

{'SHADDA_DAMMA': 0, 'SUKUN': 1, 'SHADDA_FATHA': 2, 'SHADDA_KASRATAN': 3, '_': 4, 'FATHATAN': 5, 'FATHA': 6, 'SHADDA_DAMMATAN': 7, 'DAMMA': 8, 'KASRATAN': 9, 'DAMMATAN': 10, 'SHADDA_KASRA': 11, 'SHADDA_FATHATAN': 12, 'KASRA': 13, 'SHADDA': 14}


In [9]:
# Exploring information about the data
print('The number of outputs is tag_map', len(tag_map))
# The number of vocabulary tokens (including <PAD>)
g_vocab_size = len(vocab)
print(f"Num of vocabulary words: {g_vocab_size}")
print('The vocab size is', len(vocab))
print('The training size is', t_size)
print('The validation size is', v_size)
print('An example of the first sentence is', t_sentences[0])
print('An example of its corresponding label is', t_labels[0])
len(t_sentences[0])==len( t_labels[0])

The number of outputs is tag_map 15
Num of vocabulary words: 108279
The vocab size is 108279
The training size is 116499
The validation size is 5919
An example of the first sentence is [7132, 87861, 12410, 75463, 8170, 69966, 92358, 22261, 103267, 13969, 7132, 9121, 66367, 56198, 9838, 88952, 64143, 87134, 76449, 1094, 78637, 56937, 8145, 31582, 47154, 34505, 24551, 91142, 90569, 103267, 13969, 62183, 103267, 93999, 87861, 83641, 14300, 107332, 23979, 104546, 27243, 80042, 72673, 84342, 72817, 36181, 79763, 80865, 50362, 68288, 60149, 62183, 107230, 46277, 87032, 41773, 87235, 55787, 54740, 92358, 107230, 107332, 57484, 106726, 106045, 14072, 102383, 32237, 93808, 53442]
An example of its corresponding label is [8, 1, 6, 8, 8, 1, 6, 0, 8, 6, 8, 9, 4, 13, 13, 9, 13, 6, 4, 6, 8, 1, 13, 5, 13, 9, 9, 11, 9, 8, 6, 8, 13, 9, 1, 9, 8, 6, 13, 13, 13, 13, 4, 13, 13, 13, 13, 13, 6, 9, 10, 8, 9, 13, 2, 6, 10, 13, 4, 6, 10, 6, 13, 4, 6, 6, 13, 6, 1, 1]


True

# NERDataset
The class that impelements the dataset for NER

In [10]:
class NERDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad):
    """
    This is the constructor of the NERDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    #####################  create two tensors one for x and the other for labels ###############################
    self.x = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in x], padding_value=pad,batch_first = True)
    self.y = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in y], padding_value=10,batch_first = True)
    #################################################################################################################
    # print the max length of the sentences
    print('The max length of the sentences is', self.x.shape[1])
    print('The max length of the labels is', self.y.shape[1])
  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    ######################  return the length of the dataset #############################
    return len(self.x)
    ###########################################################################################

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    ######################  return a tuple of x and y ###################################
    return self.x[idx], self.y[idx]
    ##########################################################################################

In [11]:
batch_size = 5
mini_sentences = t_sentences[0: 8]
mini_labels = t_labels[0: 8]
# print(mini_labels)
mini_dataset = NERDataset(mini_sentences, mini_labels, vocab['<PAD>'])
dummy_dataloader = torch.utils.data.DataLoader(mini_dataset, batch_size=5)
dg = iter(dummy_dataloader)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

The max length of the sentences is 70
The max length of the labels is 70
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([3, 70]) torch.Size([3, 70])
tensor([  7132,  87861,  12410,  75463,   8170,  69966,  92358,  22261, 103267,
         13969,   7132,   9121,  66367,  56198,   9838,  88952,  64143,  87134,
         76449,   1094,  78637,  56937,   8145,  31582,  47154,  34505,  24551,
         91142,  90569, 103267,  13969,  62183, 103267,  93999,  87861,  83641,
         14300, 107332,  23979, 104546,  27243,  80042,  72673,  84342,  72817,
         36181,  79763,  80865,  50362,  68288,  60149,  62183, 107230,  46277,
         87032,  41773,  87235,  55787,  54740,  92358, 107230, 107332,  57484,
        106726, 106045,  14072, 102383,  32237,  93808,  53442]) 
 tensor([ 8,  1,  6,  8,  8,  1,  6,  0,  8,  6,  8,  9,  4, 13, 13,  9, 13,  6,
         4,  6,  8,  1, 13,  5, 13,  9,  9, 11,  9,  8,  6,  8, 13,  9,  1,  9,
         8,  6, 13, 13, 13, 13,  4, 13, 13, 13, 13, 13,  6, 

# NER
The class that implementss the pytorch model for NER

In [12]:
class NER(nn.Module):
  def __init__(self, vocab_size=len(t_sentences) + len(test_sentences) + len(v_sentences), embedding_dim=300, hidden_size=50, n_classes=len(tag_map)):
    """
    The constructor of our NER model
    Inputs:
    - vacab_size: the number of unique words
    - embedding_dim: the embedding dimension
    - n_classes: the number of final classes (tags)
    """
    super(NER, self).__init__()
    #######################  Create the layers of your model #######################################
    # (1) Create the embedding layer
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
    self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)

    # (3) Create a linear layer with number of neorons = n_classes
    self.linear = nn.Linear(hidden_size, n_classes)
    #####################################################################################################

  def forward(self, sentences):
    """
    This function does the forward pass of our model
    Inputs:
    - sentences: tensor of shape (batch_size, max_length)

    Returns:
    - final_output: tensor of shape (batch_size, max_length, n_classes)
    """

    final_output = None
    #########################  implement the forward pass ####################################
    embeddings = self.embedding(sentences)
    lstm_out, (a,b) = self.lstm(embeddings)
    # print("lstm_out.size",lstm_out.size())
    # print("a.size",a.size())
    # print("b.size",b.size())
    final_output = self.linear(lstm_out)
    ###############################################################################################
    return final_output

In [13]:
model = NER()
print(model)

NER(
  (embedding): Embedding(128550, 300)
  (lstm): LSTM(300, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=15, bias=True)
)


# Training

In [14]:
def train(model, train_dataset, batch_size=512, epochs=5, learning_rate=0.01):
  """
  This function implements the training logic
  Inputs:
  - model: the model ot be trained
  - train_dataset: the training set of type NERDataset
  - batch_size: integer represents the number of examples per step
  - epochs: integer represents the total number of epochs (full training pass)
  - learning_rate: the learning rate to be used by the optimizer
  """

  ##############################  replace the Nones in the following code ##################################

  # (1) create the dataloader of the training set (make the shuffle=True)
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # (2) make the criterion cross entropy loss
  criterion = torch.nn.CrossEntropyLoss()

  # (3) create the optimizer (Adam)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader):

      # (4) move the train input to the device
      train_label = train_label.to(device)

      # (5) move the train label to the device
      train_input = train_input.to(device)


      # (6) do the forward pass
      output = model(train_input)
      print("output.shape",output.shape)
      print("train_label.shape",train_label.shape)
      # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
      batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

      # (8) append the batch loss to the total_loss_train
      total_loss_train += batch_loss

      # (9) calculate the batch accuracy (just add the number of correct predictions)
      acc = (output.argmax(2) == train_label).sum().item()

      total_acc_train += acc

      # (10) zero your gradients
      optimizer.zero_grad()


      # (11) do the backward pass
      batch_loss.backward()


      # (12) update the weights with your optimizer
      optimizer.step()

    # epoch loss
    epoch_loss = total_loss_train / len(train_dataset)

    # (13) calculate the accuracy
    epoch_acc = total_acc_train / (len(train_dataset) * train_dataset[0][0].shape[0])
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')

  ##############################################################################################################

In [15]:
train_dataset = NERDataset(t_sentences, t_labels, vocab['<PAD>'])
val_dataset = NERDataset(v_sentences, v_labels, vocab['<PAD>'])
test_dataset = NERDataset(test_sentences, test_labels, vocab['<PAD>'])

The max length of the sentences is 477
The max length of the labels is 477
The max length of the sentences is 329
The max length of the labels is 329
The max length of the sentences is 498
The max length of the labels is 498


In [16]:
train(model, train_dataset)

  0%|          | 0/228 [00:00<?, ?it/s]

output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])


  1%|▏         | 3/228 [00:01<01:25,  2.64it/s]

output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])


  3%|▎         | 6/228 [00:01<00:40,  5.46it/s]

...

output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])


 98%|█████████▊| 224/228 [00:18<00:00, 12.34it/s]

output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])


100%|██████████| 228/228 [00:18<00:00, 12.21it/s]

output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([512, 477, 15])
train_label.shape torch.Size([512, 477])
output.shape torch.Size([275, 477, 15])
train_label.shape torch.Size([275, 477])
Epochs: 5 | Train Loss: 1.373109898850089e-05         | Train Accuracy: 0.9977096824307595



# Evaluation

In [17]:
def evaluate(model, test_dataset, batch_size=512):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ###########################  Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  total_acc_test = 0

  # (2) disable gradients
  with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)

      # (5) do the forward pass
      output = model(test_input)

      # accuracy calculation (just add the correct predicted items to total_acc_test)
      acc = (output.argmax(2) == test_label).sum().item()
      total_acc_test += acc

    # (6) calculate the over all accuracy
    total_acc_test /= (len(test_dataset) * test_dataset[0][0].shape[0])
  ##################################################################################################


  print(f'\nTest Accuracy: {total_acc_test}')

In [18]:
evaluate(model, test_dataset)

100%|██████████| 12/12 [00:00<00:00, 43.99it/s]


Test Accuracy: 0.9980047391130078


In [ ]:
torch.save(model.state_dict(), f'./SavedModels/model_lstm_linearNN')